In [5]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0,
    groq_api_key='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',
    model_name="llama-3.3-70b-versatile"
)
response = llm.invoke(" who is the rocket man of india")
print(response.content)
    
    

The "Rocket Man of India" is a nickname given to Dr. A.P.J. Abdul Kalam, a renowned Indian scientist, engineer, and former President of India. He is often referred to as the "Missile Man of India" or "Rocket Man of India" due to his significant contributions to India's space and missile programs.

Dr. Kalam played a crucial role in the development of India's first satellite launch vehicle, SLV-3, and the first indigenous guided missile, Prithvi. He also worked on the development of the Agni and Trishul missiles, which are key components of India's defense system.

As a scientist and engineer, Dr. Kalam worked at the Indian Space Research Organisation (ISRO) and the Defence Research and Development Organisation (DRDO), where he made significant contributions to the development of India's space and missile programs. He was also a strong advocate for the use of technology to improve the lives of people, particularly in rural areas.

Dr. Kalam's achievements and contributions to India's sp

In [11]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-47078?from=job%20search%20funnel")
page_data = loader.load().pop().page_content
print(page_data)

Apply for Senior Product Analyst

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsData & AnalyticsSenior Product AnalystHilversum, North HollandBecome a Part of the NIKE, Inc. Team
NIKE,

In [19]:
from langchain_core.prompts import PromptTemplate
prompt_extract = PromptTemplate.from_template(
    """
    ###SCRAPED TEXT FROM WEBSITE
    {page_data}
    ###INSTRUCTION
    The scraped text is from career's page of the website.
    Your job is to extract the job postings and  return them in JSIN format containing
    following keys: `role`, `experience`,`skills` and `description`.
    Only return the vaild JSON
    ###VALID JSON(NO PREAMBLE):
    """
)
chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data':page_data})
print(res.content)

```json
{
  "role": "Senior Product Analyst",
  "experience": "Proven track record in Product Analytics role within a professional environment turning data into insights",
  "skills": [
    "Data Modelling/Data Architectures",
    "SQL",
    "NoSQL",
    "RDBMS",
    "Data warehouse architectures",
    "Visualization tools such as Cognos, Tableau",
    "Writing and maintaining User & Technical Documentation"
  ],
  "description": "We’re looking for a Senior Product Analyst to join our Nike Enterprise Data & AI team to work within our squads to develop and deliver data, analytical and AI/ML products that help Nike bring inspiration and innovation to every athlete* in the world."
}
```


In [21]:
type(res.content)

str

In [29]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Senior Product Analyst',
 'experience': 'Proven track record in Product Analytics role within a professional environment turning data into insights',
 'skills': ['Data Modelling/Data Architectures',
  'SQL',
  'NoSQL',
  'RDBMS',
  'Data warehouse architectures',
  'Visualization tools such as Cognos, Tableau',
  'Writing and maintaining User & Technical Documentation'],
 'description': 'We’re looking for a Senior Product Analyst to join our Nike Enterprise Data & AI team to work within our squads to develop and deliver data, analytical and AI/ML products that help Nike bring inspiration and innovation to every athlete* in the world.'}

In [31]:
type(json_res)

dict

In [33]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [37]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")
if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links":row["Links"]},
                       ids=[str(uuid.uuid4())])

In [39]:
links = collection.query(query_texts=["Experience in Python", "Expertise in React"], n_results=2).get('metadatas')
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/react-native-portfolio'}]]

In [41]:
job = json_res
job['skills']

['Data Modelling/Data Architectures',
 'SQL',
 'NoSQL',
 'RDBMS',
 'Data warehouse architectures',
 'Visualization tools such as Cognos, Tableau',
 'Writing and maintaining User & Technical Documentation']

In [51]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Yaswanth Veguru, AI/ML Engineer at TEMS Tech Solutions. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
)
chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list":links})
print(res.content)

Subject: Expert Product Analytics Solutions for Nike Enterprise Data & AI Team

Dear Hiring Manager,

I came across the Senior Product Analyst role at Nike and was impressed by the company's commitment to leveraging data and AI to drive innovation. As a business development executive at AtliQ, an AI & Software Consulting company, I believe our expertise can help fulfill your team's needs.

At AtliQ, we have a proven track record of empowering enterprises with tailored solutions, fostering scalability, process optimization, cost reduction, and heightened overall efficiency. Our team of experts has extensive experience in developing and delivering data, analytical, and AI/ML products that drive business growth.

In particular, our capabilities align with the requirements of the Senior Product Analyst role, including:

* Data Modelling/Data Architectures
* SQL, NoSQL, and RDBMS
* Data warehouse architectures
* Visualization tools such as Cognos and Tableau
* Writing and maintaining User &